# The K-Secretary Problem and Optimal Thresholds

The problem setup, definitions, LP formulation, and its interpretation are from Buchbinder Jain Singh '14 and Chan Chen Jiang '15.

## Problem Setup

**Input:**
- $n$ items arrive in uniformly random order
- Items have a total ordering (only relative comparisons observable)
- Algorithm has $K$ quotas for selecting items
- Decisions are **irrevocable** (made upon arrival)

**Goal:** Maximize $\mathbb{E}[\text{number of selected items among the } K \text{ best overall}]$

**Performance ratio:** $\frac{\mathbb{E}[\text{number selected among } K \text{ best}]}{K}$

## Key Definitions

**k-potential:** An item arriving at step $i$ is a **k-potential** if it ranks k-th among all items arrived so far (including itself).

**k≥-potential:** An item is a **k≥-potential** if it is a $k'$-potential for some $k' \leq k$.

**Quotas:** $Q_K, Q_{K-1}, \ldots, Q_1$ (used in order: $Q_K$ first, $Q_1$ last)

## Optimal K-Threshold Algorithm

**Parameters:** $K^2$ thresholds $\{\tau_{j,k} : j \in [K], k \in [K]\}$ satisfying:
1. For each quota $j$: $0 < \tau_{j,1} \leq \tau_{j,2} \leq \cdots \leq \tau_{j,K} \leq 1$
2. For each potential level $k$: $0 < \tau_{K,k} \leq \tau_{K-1,k} \leq \cdots \leq \tau_{1,k} \leq 1$

**Selection rule:** 
- After time $\tau_{j,k}$, quota $Q_j$ may be used for any $k$≥-potential
- Select greedily: whenever an item can be selected, use the highest-indexed available quota

**Intuition:** Each quota "matures" progressively (condition 1), and higher-indexed quotas mature earlier (condition 2).

## Linear Programming Formulation: $\text{LP}_n(K,K)$

### Decision Variables

$$z_{j|k}(i) = \Pr[\text{item at step } i \text{ selected using quota } Q_j \mid \text{it is a } k\text{-potential}]$$

### The Linear Program

$$
\begin{align}
\max \quad & \sum_{j=1}^K \sum_{k=1}^K \sum_{i=1}^n \frac{1}{n} \sum_{\ell=k}^K \delta_{k|\ell}(i) \cdot z_{j|k}(i) \\
\text{s.t.} \quad & z_{j|k}(i) \leq \sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K \left[z_{(j+1)|\ell}(m) - z_{j|\ell}(m)\right], 
\quad \forall i \in [n], k \in [K], 1 \leq j < K \\
& z_{K|k}(i) \leq 1 - \sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K z_{K|\ell}(m), 
\quad \forall i \in [n], k \in [K] \\
& z_{j|k}(i) \geq 0, \quad \forall i \in [n], j, k \in [K] \\
\end{align}
$$

where

$$\delta_{k|\ell}(i) = \frac{\binom{n-i}{\ell-k}\binom{i-1}{k-1}}{\binom{n-1}{\ell-1}}$$

## Interpretation

### The $\delta$ Coefficients

$$\delta_{k|\ell}(i) = \Pr[\text{item at step } i \text{ is a } k\text{-potential} \mid \text{it is the } \ell\text{-th best overall}]$$

**Explanation:** Given the $\ell$-th best item arrives at step $i$, it is a $k$-potential iff exactly $(k-1)$ of the $(\ell-1)$ better items arrive in the $(i-1)$ steps before it.

### Objective Function

Define auxiliary quantities:
- $\gamma_k(i) := \sum_{j=1}^K z_{j|k}(i) = \Pr[\text{item at step } i \text{ selected} \mid \text{it is a } k\text{-potential}]$
- $\beta_\ell(i) := \sum_{k=1}^\ell \delta_{k|\ell}(i) \cdot \gamma_k(i) = \Pr[\ell\text{-th best item selected} \mid \text{it arrives at step } i]$

The objective equals:
$$\sum_{\ell=1}^K \underbrace{\sum_{i=1}^n \frac{1}{n} \beta_\ell(i)}_{\Pr[\ell\text{-th best selected}]}$$

**Interpretation:** Sum over all $K$ best items of the probability each is selected (since each arrives at step $i$ with probability $1/n$).

### Constraints

$$\sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K z_{j|\ell}(m) = \Pr[\text{quota } Q_j \text{ used by step } i]$$

**Explanation:** At each step $m < i$, an $\ell$-potential arrives with probability $1/m$, and quota $Q_j$ is used with probability $z_{j|\ell}(m)$.

The RHS of constraints gives:
- $\sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K [z_{(j+1)|\ell}(m) - z_{j|\ell}(m)] = \Pr[\text{exactly } j \text{ quotas remain at step } i]$ for $j < K$
- $1 - \sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K z_{K|\ell}(m) = \Pr[\text{all } K \text{ quotas remain at step } i]$ for $j = K$

**Constraint meaning:** To select with quota $Q_j$ at step $i$, that quota must be available.

**Theorem:** Any K-secretary algorithm induces feasible $z$ values with objective value equal to $\mathbb{E}[\text{number selected among } K \text{ best}]$.

**Corollary:** The optimal LP value equals the optimal performance ratio achievable by any algorithm.

# Demo: A Framework for Analyzing the Class of Algorithms

In [15]:
import numpy as np
from scipy.special import comb
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import matplotlib.pyplot as plt
from itertools import combinations
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

In [16]:
def compute_delta(n, i, k, ell):
    """
    Compute delta_{k|ell}(i) = P[item at step i is k-potential | it's ell-th best overall]
    """
    if i < 1 or k < 1 or ell < k or k > i:
        return 0.0
    
    numerator = comb(n - i, ell - k, exact=True) * comb(i - 1, k - 1, exact=True)
    denominator = comb(n - 1, ell - 1, exact=True)
    
    return numerator / denominator if denominator > 0 else 0.0

In [17]:
def create_k_secretary_lp_model(n, K, verbose=False):
    """
    Creates a Pyomo ConcreteModel for the base K-secretary LP.
    Does not solve.
    """
    if verbose:
        print(f"Building Pyomo model components for n={n}, K={K}...")

    # Precompute delta
    delta = np.zeros((n, K, K))
    for i in range(1, n + 1):
        for k in range(1, K + 1):
            for ell in range(k, K + 1):
                delta[i-1, k-1, ell-1] = compute_delta(n, i, k, ell)
    
    model = pyo.ConcreteModel()

    # --- Indices ---
    model.J = pyo.Set(initialize=range(K)) # Quotas
    model.K = pyo.Set(initialize=range(K)) # Potentials
    model.I = pyo.Set(initialize=range(n)) # Steps
    model.VARS = model.J * model.K * model.I

    # --- Variables ---
    # z[j, k, i] (using 0-based indexing)
    model.z = pyo.Var(model.VARS, bounds=(0.0, 1.0))

    # --- Objective Function ---
    c_dict = {}
    for j, k, i in model.VARS:
        c_val = 0.0
        for ell in range(k, K): # ell is 0-indexed k..K-1
            c_val += (1.0 / n) * delta[i, k, ell]
        c_dict[(j,k,i)] = c_val
    
    model.objective = pyo.Objective(
        expr=sum(c_dict[v] * model.z[v] for v in model.VARS),
        sense=pyo.maximize
    )

    # --- Constraints ---
    model.lp_constrs = pyo.ConstraintList()

    # Constraints for j < K (j_idx = 0...K-2)
    for j in range(K - 1):
        for k in range(K):
            for i in range(1, n): # i_idx = 1...n-1
                rhs = 0
                for m in range(i): # m_idx = 0...i-1
                    for ell in range(K):
                        rhs += (1.0 / (m + 1)) * (model.z[j+1, ell, m] - model.z[j, ell, m])
                model.lp_constrs.add(model.z[j, k, i] <= rhs)
    
    # Constraints for j = K (j_idx = K-1)
    j = K - 1
    for k in range(K):
        for i in range(1, n): # i_idx = 1...n-1
            rhs = 1.0
            for m in range(i): # m_idx = 0...i-1
                for ell in range(K):
                    rhs -= (1.0 / (m + 1)) * model.z[j, ell, m]
            model.lp_constrs.add(model.z[j, k, i] <= rhs)

    if verbose:
        print("Base model components built.")
        
    return model

In [18]:
def solve_base_lp_pyomo(n, K, verbose=True):
    """
    Solves the base K-secretary LP (B=0) using Pyomo.
    """
    # Create the base model
    model = create_k_secretary_lp_model(n, K, verbose=verbose)

    if verbose:
        print("Base model created. Solving with Gurobi...")

    # Solve
    opt = SolverFactory('gurobi')
    results = opt.solve(model, tee=False) # tee=True to see solver logs
    
    obj_val = 0.0
    if results.solver.termination_condition == pyo.TerminationCondition.optimal:
        obj_val = pyo.value(model.objective)
        if verbose:
            print(f"\n{'='*60}\nOPTIMAL SOLUTION FOUND (B=0)\n{'='*60}")
            print(f"Expected # of top-{K} items selected: {obj_val:.6f}")
            print(f"Competitive ratio: {obj_val / K:.6f}")
            print(f"{'='*60}\n")
    else:
        print(f"Base LP solve failed: {results.solver.termination_condition}")
        return None, None

    return model, obj_val

In [19]:
# Run for n=30, K=2 with Pyomo
n_demo = 30
K_demo = 2

print(f"Solving K-secretary problem with n={n_demo}, K={K_demo} (Pyomo B=0)")
print(f"This is the exact LP from equations (1)-(4)\n")

# We get the solved model and the base objective value
base_model, obj_val_base = solve_base_lp_pyomo(n_demo, K_demo, verbose=True)

Solving K-secretary problem with n=30, K=2 (Pyomo B=0)
This is the exact LP from equations (1)-(4)

Building Pyomo model components for n=30, K=2...
Base model components built.
Base model created. Solving with Gurobi...

OPTIMAL SOLUTION FOUND (B=0)
Expected # of top-2 items selected: 1.013048
Competitive ratio: 0.506524



The LP solution finds a single, fully optimal algorithm. We can, however, analyze the *class* of ordinal algorithms by parameterizing them.

We can formalize this by introducing an external agent ("nature") that has a "budget" $B$. This budget allows the agent to choose $B$ of the algorithm's decision variables and fix them to specific values $\vec{c}$. This defines a subclass of algorithms, allowing us to analyze the performance spectrum.

**Note:** For this analysis, we restrict to $\vec{c} = 0$ (fixing variables to zero).

## Mathematical Formulation

Let $I$ be the set of all variable indices $v = (j, k, i)$.
Let $z = \{z_v\}_{v \in I}$ be the vector of all decision variables $z_{j|k}(i)$.
Let $c_v$ be the objective coefficient for variable $z_v$.

The objective is:
$$\text{Obj}(z) = \sum_{v \in I} c_v z_v$$

Let $\text{LP-Constraints}(z)$ represent the full set of original LP constraints.

### Performance Function

Introduce a binary vector $y = \{y_v\}_{v \in I}$ where $y_v=1$ means variable $v$ is fixed to 0.

The function $Q(y)$ quantifies the *algorithm's best possible performance* given nature's choice $y$:

$$Q(y) = \max_{z} \quad \text{Obj}(z)$$
$$\text{s.t.} \quad \text{LP-Constraints}(z)$$
$$z_v = 0 \quad \forall v \text{ with } y_v = 1$$

### Best- and Worst-Case Performance

With a fixed budget $B$ (i.e., $\sum_{v \in I} y_v = B$):

**Best-case (max-max):**
$$V_{\text{best}}(B) = \max_{y: \sum y_v = B} Q(y) = V_{\text{best}}(0)$$
*(Nature cannot improve beyond optimal; the maximum is achieved with no variables fixed)*

**Worst-case (min-max):**
$$V_{\text{worst}}(B) = \min_{y: \sum y_v = B} Q(y)$$
*(Nature chooses which $B$ variables to fix to 0 to minimize performance)*

### Interference Spectrum with Parameter $\alpha$

For a given interference tolerance $\alpha \in [0, 1]$, we solve:

**Threshold problem:**
$$\min_{y: \sum y_v = B} Q(y)$$
$$\text{s.t.} \quad Q(y) \geq (1-\alpha) V_{\text{best}}(B) + \alpha V_{\text{worst}}(B)$$

This finds the **worst-case configuration** among all configurations meeting the $\alpha$-parameterized threshold.

**Interpretation:**
- $\alpha = 0$: Threshold = $V_{\text{best}}$ → only optimal algorithm satisfies this
- $\alpha = 1$: Threshold = $V_{\text{worst}}$ → worst-case algorithm satisfies this  
- $\alpha \in (0,1)$: Threshold interpolates → explores intermediate configurations

This generates a **continuous spectrum** showing how performance degrades with interference.

## Computing $V_{\text{best}}$ and $V_{\text{worst}}$

For a fixed budget $B$:
- $V_{\text{best}}(B) = V_{\text{best}}(0)$ = optimal LP value (nature cannot improve beyond optimal)
- $V_{\text{worst}}(B)$ = minimum performance when nature fixes $B$ variables to 0

We use **enumeration** to compute $V_{\text{worst}}(B)$ exactly by trying all $\binom{|\text{vars}|}{B}$ combinations.

---

## Interactive Analysis Setup

The cells below create an interactive interface for exploring different configurations.

In [20]:
# ================================
# HELPER FUNCTIONS
# ================================

def get_meaningful_vars(n, K, coeff_threshold=1e-3):
    """Identify variables with non-negligible objective coefficients."""
    delta = np.zeros((n, K, K))
    for i in range(1, n + 1):
        for k in range(1, K + 1):
            for ell in range(k, K + 1):
                delta[i-1, k-1, ell-1] = compute_delta(n, i, k, ell)
    
    c_dict = {}
    for j in range(K):
        for k in range(K):
            for i in range(n):
                c_val = 0.0
                for ell in range(k, K):
                    c_val += (1.0 / n) * delta[i, k, ell]
                c_dict[(j, k, i)] = c_val
    
    meaningful_vars = []
    for j in range(K):
        for k in range(K):
            for i in range(n):
                if i < k or i == 0:
                    continue
                if abs(c_dict[(j, k, i)]) > coeff_threshold:
                    meaningful_vars.append((j, k, i))
    
    return meaningful_vars, c_dict


def create_k_secretary_lp_model_restricted(n, K, meaningful_vars, verbose=False):
    """Creates LP with only meaningful variables (non-meaningful implicitly 0)."""
    delta = np.zeros((n, K, K))
    for i in range(1, n + 1):
        for k in range(1, K + 1):
            for ell in range(k, K + 1):
                delta[i-1, k-1, ell-1] = compute_delta(n, i, k, ell)
    
    model = pyo.ConcreteModel()
    model.VARS = pyo.Set(initialize=meaningful_vars)
    model.z = pyo.Var(model.VARS, bounds=(0.0, 1.0))

    # Objective
    c_dict = {}
    for j, k, i in model.VARS:
        c_val = 0.0
        for ell in range(k, K):
            c_val += (1.0 / n) * delta[i, k, ell]
        c_dict[(j,k,i)] = c_val
    
    model.objective = pyo.Objective(
        expr=sum(c_dict[v] * model.z[v] for v in model.VARS),
        sense=pyo.maximize
    )

    # Constraints (only for meaningful variables)
    model.lp_constrs = pyo.ConstraintList()

    # Constraints for j < K
    for j in range(K - 1):
        for k in range(K):
            for i in range(1, n):
                if (j, k, i) not in meaningful_vars:
                    continue
                
                rhs = 0
                for m in range(i):
                    for ell in range(K):
                        if (j+1, ell, m) in meaningful_vars:
                            rhs += (1.0 / (m + 1)) * model.z[(j+1, ell, m)]
                        if (j, ell, m) in meaningful_vars:
                            rhs -= (1.0 / (m + 1)) * model.z[(j, ell, m)]
                
                model.lp_constrs.add(model.z[(j, k, i)] <= rhs)
    
    # Constraints for j = K
    j = K - 1
    for k in range(K):
        for i in range(1, n):
            if (j, k, i) not in meaningful_vars:
                continue
            
            rhs = 1.0
            for m in range(i):
                for ell in range(K):
                    if (j, ell, m) in meaningful_vars:
                        rhs -= (1.0 / (m + 1)) * model.z[(j, ell, m)]
            
            model.lp_constrs.add(model.z[(j, k, i)] <= rhs)
        
    return model


def solve_v_worst_enumeration(n, K, B, vars_to_enumerate, coeff_threshold=1e-3):
    """
    Solve V_worst(B) by enumerating all C(|vars_to_enumerate|, B) combinations.
    Nature fixes B variables to 0 (the worst case).
    """
    v_worst = float('inf')
    worst_vars = None
    opt = SolverFactory('gurobi')
    
    # Get ALL meaningful vars for model building
    all_meaningful_vars, _ = get_meaningful_vars(n, K, coeff_threshold)
    
    # Try all combinations of B variables to fix to 0
    for vars_to_fix in combinations(vars_to_enumerate, B):
        model = create_k_secretary_lp_model_restricted(n, K, all_meaningful_vars)
        
        # Fix selected variables to 0
        for v in vars_to_fix:
            model.z[v].fix(0.0)
        
        results = opt.solve(model, tee=False)
        
        if results.solver.termination_condition == pyo.TerminationCondition.optimal:
            obj_val = pyo.value(model.objective)
            if obj_val < v_worst:
                v_worst = obj_val
                worst_vars = vars_to_fix
        
        for v in vars_to_fix:
            model.z[v].unfix()
    
    return v_worst, list(worst_vars) if worst_vars else []


# ================================
# INTERACTIVE ANALYSIS SETUP
# ================================

# Get all meaningful variables (computed once)
all_meaningful_vars, c_dict = get_meaningful_vars(n_demo, K_demo)

# Create output widget
output_widget = widgets.Output()

# Create interactive widgets
quota_widget = widgets.Dropdown(
    options=[('Q_1 (Quota 1)', 0), ('Q_2 (Quota 2)', 1)],
    value=1,
    description='Quota:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

k_potential_widget = widgets.Dropdown(
    options=[('1-potential', 0), ('2-potential', 1)],
    value=0,
    description='K-potential:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

budget_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    step=1,
    description='Budget B:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

run_button = widgets.Button(
    description='Run Analysis',
    button_style='primary',
    icon='play',
    layout=widgets.Layout(width='150px')
)

@output_widget.capture(clear_output=True)
def run_analysis(b):
    quota_idx = quota_widget.value
    k_pot_idx = k_potential_widget.value
    B = budget_widget.value
    
    # Filter variables
    vars_to_enum = [v for v in all_meaningful_vars 
                    if v[0] == quota_idx and v[1] == k_pot_idx]
    
    print(f"Configuration: Quota Q_{quota_idx+1}, {k_pot_idx+1}-potentials, Budget B={B}")
    print(f"Variables to enumerate: {len(vars_to_enum)} (from {len(all_meaningful_vars)} total)")
    print(f"-" * 60)
    
    # Check if feasible
    if len(vars_to_enum) < B:
        print(f"\n❌ ERROR: Budget B={B} exceeds available variables ({len(vars_to_enum)})")
        print("   Please reduce the budget or change the filter.")
        return
    
    # Solve V_worst
    print(f"\nSolving V_worst({B})... (trying {comb(len(vars_to_enum), B, exact=True)} combinations)")
    v_worst, worst_vars = solve_v_worst_enumeration(n_demo, K_demo, B, vars_to_enum)
    
    # Display results
    print(f"\n{'='*60}")
    print(f"RESULTS")
    print(f"{'='*60}")
    print(f"V_best({B})  = {obj_val_base:.6f}  (CR = {obj_val_base/K_demo:.6f})")
    print(f"V_worst({B}) = {v_worst:.6f}  (CR = {v_worst/K_demo:.6f})")
    print(f"\nPerformance gap: {(obj_val_base - v_worst)/obj_val_base * 100:.2f}%")
    
    if worst_vars:
        print(f"\nWorst configuration (fixing {B} variable{'s' if B > 1 else ''} to 0):")
        for v in worst_vars:
            j, k, i = v
            print(f"  • z[{j},{k},{i}] = 0  →  Q_{j+1}, {k+1}-potential, step {i+1}  (coeff: {c_dict[v]:.6f})")
    print(f"{'='*60}")

run_button.on_click(run_analysis)

In [21]:
# Display interface (can be run multiple times without duplication)
print("="*60)
print("INTERACTIVE INTERFERENCE ANALYSIS")
print("="*60)
print(f"\nProblem: n={n_demo}, K={K_demo}")
print(f"V_best = {obj_val_base:.6f} (CR = {obj_val_base/K_demo:.6f})")
print(f"Meaningful variables: {len(all_meaningful_vars)}\n")

display(widgets.VBox([
    widgets.HTML("<b>Select which variables nature can interfere with:</b>"),
    widgets.HBox([quota_widget, k_potential_widget, budget_widget]),
    run_button,
    output_widget
], layout=widgets.Layout(padding='10px')))

INTERACTIVE INTERFERENCE ANALYSIS

Problem: n=30, K=2
V_best = 1.013048 (CR = 0.506524)
Meaningful variables: 116



## Interactive Analysis: Interference Spectrum

We now explore the full spectrum of algorithm behavior under interference. Using the interference parameter $\alpha \in [0,1]$, we identify which variables nature should fix to achieve different levels of performance degradation.

In [22]:
# ================================
# INTERFERENCE SPECTRUM SETUP
# ================================

# Create output widget
spectrum_output_widget = widgets.Output()

# Create interactive widgets for spectrum analysis
spectrum_quota_widget = widgets.Dropdown(
    options=[('Q_1 (Quota 1)', 0), ('Q_2 (Quota 2)', 1)],
    value=1,
    description='Quota:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

spectrum_k_potential_widget = widgets.Dropdown(
    options=[('1-potential', 0), ('2-potential', 1)],
    value=0,
    description='K-potential:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

spectrum_budget_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    step=1,
    description='Budget B:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

alpha_step_widget = widgets.FloatSlider(
    value=0.04,
    min=0.01,
    max=0.1,
    step=0.01,
    description='α step:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

run_spectrum_button = widgets.Button(
    description='Generate Spectrum',
    button_style='success',
    icon='chart-line',
    layout=widgets.Layout(width='180px')
)

@spectrum_output_widget.capture(clear_output=True)
def run_spectrum_analysis(b):
    quota_idx = spectrum_quota_widget.value
    k_pot_idx = spectrum_k_potential_widget.value
    B = spectrum_budget_widget.value
    alpha_step = alpha_step_widget.value
    
    # Filter variables
    vars_to_enum = [v for v in all_meaningful_vars 
                    if v[0] == quota_idx and v[1] == k_pot_idx]
    
    print(f"Configuration: Quota Q_{quota_idx+1}, {k_pot_idx+1}-potentials, Budget B={B}")
    print(f"Filtered variables: {len(vars_to_enum)} | α step: {alpha_step}")
    print(f"=" * 60)
    
    if len(vars_to_enum) < B:
        print(f"\n❌ ERROR: Budget B={B} exceeds available variables ({len(vars_to_enum)})")
        print("   Please reduce the budget or change the filter.")
        return
    
    # Precompute all performances
    from scipy.special import comb as scipy_comb
    total_combos = int(scipy_comb(len(vars_to_enum), B, exact=True))
    print(f"\nStep 1/3: Precomputing {total_combos} performance values...")
    print(f"  Variables to enumerate: {len(vars_to_enum)}, Budget B: {B}")
    
    all_performances = []
    opt = SolverFactory('gurobi')
    
    for i, vars_to_fix in enumerate(combinations(vars_to_enum, B)):
        if i % 10 == 0:
            print(f"  Progress: {i}/{total_combos}", end='\r')
        
        model = create_k_secretary_lp_model_restricted(n_demo, K_demo, all_meaningful_vars)
        
        for v in vars_to_fix:
            model.z[v].fix(0.0)
        
        results = opt.solve(model, tee=False)
        
        if results.solver.termination_condition == pyo.TerminationCondition.optimal:
            performance = pyo.value(model.objective)
            all_performances.append((performance, vars_to_fix))
        
        for v in vars_to_fix:
            model.z[v].unfix()
    
    # Sort by performance
    all_performances.sort(key=lambda x: x[0])
    v_worst = all_performances[0][0]
    v_best = obj_val_base
    
    print(f"\n  ✓ Computed {len(all_performances)} performances")
    print(f"  Performance range: [{v_worst:.6f}, {v_best:.6f}]")
    print(f"  Gap from optimal: {(v_best - v_worst)/v_best * 100:.2f}%")
    print(f"  Worst-case variables: {all_performances[0][1]}")
    
    # Generate α-spectrum
    print(f"\nStep 2/3: Generating α-spectrum...")
    alpha_grid = np.arange(0, 1.0 + alpha_step, alpha_step)
    
    results = {
        'alpha': [],
        'performance': [],
        'competitive_ratio': [],
        'steps_selected': [],
        'threshold': []
    }
    
    for alpha in alpha_grid:
        threshold = (1 - alpha) * v_best + alpha * v_worst
        
        # Find smallest performance >= threshold
        perf_found = None
        vars_found = None
        
        for perf, vars_fixed in all_performances:
            if perf >= threshold:
                perf_found = perf
                vars_found = vars_fixed
                break
        
        if perf_found is not None:
            results['alpha'].append(alpha)
            results['performance'].append(perf_found)
            results['competitive_ratio'].append(perf_found / K_demo)
            results['threshold'].append(threshold)
            
            # Store ALL steps from the tuple of variables
            if vars_found:
                steps = [v[2] + 1 for v in vars_found]
                results['steps_selected'].append(steps)
            else:
                results['steps_selected'].append([])
    
    print(f"  ✓ Computed {len(results['alpha'])} α values")
    
    # Create plots
    print(f"\nStep 3/3: Generating visualizations...")
    fig = plt.figure(figsize=(18, 5))
    
    # Plot 1: Competitive Ratio vs α
    ax1 = plt.subplot(1, 3, 1)
    ax1.plot(results['alpha'], results['competitive_ratio'], 'o-', linewidth=2, markersize=4, label='Worst-case CR', color='#2E86AB')
    ax1.plot(results['alpha'], [t/K_demo for t in results['threshold']], '--', alpha=0.6, label='Threshold', color='#A23B72')
    ax1.axhline(y=v_best/K_demo, color='#06A77D', linestyle=':', alpha=0.7, linewidth=2, label='V_best')
    ax1.axhline(y=v_worst/K_demo, color='#D62828', linestyle=':', alpha=0.7, linewidth=2, label='V_worst')
    ax1.set_xlabel('Interference Parameter α', fontsize=12)
    ax1.set_ylabel('Competitive Ratio', fontsize=12)
    ax1.set_title('Performance Degradation', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.legend(framealpha=0.9)
    ax1.set_xlim(-0.02, 1.02)
    
    # Plot 2: Explicit dots showing all critical variables
    ax2 = plt.subplot(1, 3, 2)
    for alpha_val, steps in zip(results['alpha'], results['steps_selected']):
        for step in steps:
            ax2.plot(alpha_val, step, 'o', markersize=6, color='#F77F00', alpha=0.7)
    
    ax2.set_xlabel('Interference Parameter α', fontsize=12)
    ax2.set_ylabel('Step i (1-indexed)', fontsize=12)
    ax2.set_title(f'Critical Variables (Explicit)', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.set_xlim(-0.02, 1.02)
    ax2.set_ylim(0, n_demo+1)
    
    # Plot 3: 1D bar chart showing step frequency across all alphas
    ax3 = plt.subplot(1, 3, 3)
    
    # Count frequency of each step across ALL alpha values
    step_frequency = np.zeros(n_demo)
    for steps in results['steps_selected']:
        for step in steps:
            step_frequency[step - 1] += 1
    
    # Normalize to get percentage
    step_frequency_pct = (step_frequency / len(results['alpha'])) * 100
    
    # Create color gradient based on frequency
    colors = plt.cm.YlOrRd(step_frequency_pct / step_frequency_pct.max())
    
    # Create horizontal bar chart
    steps_range = np.arange(1, n_demo + 1)
    bars = ax3.barh(steps_range, step_frequency_pct, color=colors, edgecolor='none')
    
    ax3.set_xlabel('Frequency (%)', fontsize=12)
    ax3.set_ylabel('Step i (1-indexed)', fontsize=12)
    ax3.set_title('Step Frequency Distribution', fontsize=14, fontweight='bold')
    ax3.set_ylim(0, n_demo + 1)
    ax3.set_xlim(0, max(step_frequency_pct) * 1.1 if step_frequency_pct.max() > 0 else 100)
    ax3.grid(True, alpha=0.2, axis='x')
    
    plt.tight_layout()
    filename = f'k-sec-interference-spectrum-Q{quota_idx+1}-k{k_pot_idx+1}-B{B}.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    print(f"  ✓ Plot saved: {filename}")
    plt.show()
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"SUMMARY STATISTICS")
    print(f"{'='*60}")
    print(f"α range:           [{min(results['alpha']):.2f}, {max(results['alpha']):.2f}]")
    print(f"Performance range: [{min(results['performance']):.6f}, {max(results['performance']):.6f}]")
    print(f"CR range:          [{min(results['competitive_ratio']):.6f}, {max(results['competitive_ratio']):.6f}]")
    
    # Get all unique steps across all alphas
    all_steps = set()
    for steps in results['steps_selected']:
        all_steps.update(steps)
    if all_steps:
        print(f"Step range:        [{min(all_steps)}, {max(all_steps)}]")
    
    print(f"Performance drop:  {(max(results['performance']) - min(results['performance']))/max(results['performance'])*100:.2f}%")
    print(f"{'='*60}")

run_spectrum_button.on_click(run_spectrum_analysis)

In [23]:
# Display interface (can be run multiple times without duplication)
print("="*60)
print("INTERFERENCE SPECTRUM ANALYSIS")
print("="*60)
print("\nGenerate the full α-spectrum showing how performance")
print("degrades as we allow more interference.\n")

display(widgets.VBox([
    widgets.HTML("<b>Select which variables nature can interfere with:</b>"),
    widgets.HBox([spectrum_quota_widget, spectrum_k_potential_widget]),
    widgets.HBox([spectrum_budget_widget, alpha_step_widget]),
    run_spectrum_button,
    spectrum_output_widget
], layout=widgets.Layout(padding='10px')))

INTERFERENCE SPECTRUM ANALYSIS

Generate the full α-spectrum showing how performance
degrades as we allow more interference.



## 3D Surface Analysis: CR as a Function of B and α

We now visualize the competitive ratio as a **3D surface** parameterized by both the budget B and the interference parameter α. This provides a complete picture of how performance degrades across the entire parameter space.

In [26]:
# ================================
# 3D SURFACE PLOT SETUP
# ================================

from mpl_toolkits.mplot3d import Axes3D

# Create output widget
surface_output_widget = widgets.Output()

# Create interactive widgets for 3D surface
surface_quota_widget = widgets.Dropdown(
    options=[('Q_1 (Quota 1)', 0), ('Q_2 (Quota 2)', 1)],
    value=1,
    description='Quota:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

surface_k_potential_widget = widgets.Dropdown(
    options=[('1-potential', 0), ('2-potential', 1)],
    value=0,
    description='K-potential:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

surface_B_max_widget = widgets.IntSlider(
    value=3,
    min=2,
    max=5,
    step=1,
    description='Max B:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

surface_alpha_step_widget = widgets.FloatSlider(
    value=0.1,
    min=0.05,
    max=0.2,
    step=0.05,
    description='α step:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='250px')
)

run_surface_button = widgets.Button(
    description='Generate 3D Surface',
    button_style='info',
    icon='cube',
    layout=widgets.Layout(width='180px')
)

@surface_output_widget.capture(clear_output=True)
def run_surface_analysis(b):
    quota_idx = surface_quota_widget.value
    k_pot_idx = surface_k_potential_widget.value
    B_max = surface_B_max_widget.value
    alpha_step = surface_alpha_step_widget.value
    
    # Filter variables
    vars_to_enum = [v for v in all_meaningful_vars 
                    if v[0] == quota_idx and v[1] == k_pot_idx]
    
    print(f"Configuration: Quota Q_{quota_idx+1}, {k_pot_idx+1}-potentials")
    print(f"Budget range: B ∈ [1, {B_max}] | α step: {alpha_step}")
    print(f"Filtered variables: {len(vars_to_enum)}")
    print(f"=" * 60)
    
    if len(vars_to_enum) < B_max:
        print(f"\n❌ ERROR: Max Budget B={B_max} exceeds available variables ({len(vars_to_enum)})")
        print(f"   Please reduce max B to at most {len(vars_to_enum)}.")
        return
    
    # Precompute V_worst for each B
    print(f"\nStep 1/2: Computing V_worst for B = 1 to {B_max}...")
    B_range = range(1, B_max + 1)
    v_worst_by_B = {}
    all_performances_by_B = {}
    
    opt = SolverFactory('gurobi')
    
    for B in B_range:
        print(f"  Computing B={B}...")
        from scipy.special import comb as scipy_comb
        total_combos = int(scipy_comb(len(vars_to_enum), B, exact=True))
        
        all_performances = []
        for i, vars_to_fix in enumerate(combinations(vars_to_enum, B)):
            if i % 50 == 0:
                print(f"    Progress: {i}/{total_combos}", end='\r')
            
            model = create_k_secretary_lp_model_restricted(n_demo, K_demo, all_meaningful_vars)
            
            for v in vars_to_fix:
                model.z[v].fix(0.0)
            
            results = opt.solve(model, tee=False)
            
            if results.solver.termination_condition == pyo.TerminationCondition.optimal:
                performance = pyo.value(model.objective)
                all_performances.append((performance, vars_to_fix))
            
            for v in vars_to_fix:
                model.z[v].unfix()
        
        all_performances.sort(key=lambda x: x[0])
        all_performances_by_B[B] = all_performances
        v_worst_by_B[B] = all_performances[0][0]
        print(f"    ✓ B={B}: V_worst = {v_worst_by_B[B]:.6f}")
    
    # Generate 3D surface: CR(B, α)
    print(f"\nStep 2/2: Generating 3D surface...")
    alpha_grid = np.arange(0, 1.0 + alpha_step, alpha_step)
    B_grid = np.array(list(B_range))
    
    # Create meshgrid
    Alpha, B_mesh = np.meshgrid(alpha_grid, B_grid)
    CR_surface = np.zeros_like(Alpha)
    
    v_best = obj_val_base
    
    for i, B in enumerate(B_range):
        v_worst = v_worst_by_B[B]
        all_performances = all_performances_by_B[B]
        
        for j, alpha in enumerate(alpha_grid):
            threshold = (1 - alpha) * v_best + alpha * v_worst
            
            # Find smallest performance >= threshold
            perf_found = v_best  # Default to v_best if no match
            for perf, _ in all_performances:
                if perf >= threshold:
                    perf_found = perf
                    break
            
            CR_surface[i, j] = perf_found / K_demo
    
    print(f"  ✓ Computed {Alpha.size} CR values")
    
    # Create 3D surface plot
    print(f"\nGenerating visualization...")
    fig = plt.figure(figsize=(14, 10))

    ax1 = fig.add_subplot(121, projection='3d')
    surf = ax1.plot_surface(
        Alpha, B_mesh, CR_surface,
        color='lightgray',   # fixed surface color (instead of colormap)
        edgecolor='none',
        alpha=0.9,
        antialiased=True
    )
    
    ax1.set_xlabel('Interference Parameter α', fontsize=11, labelpad=10)
    ax1.set_ylabel('Budget B', fontsize=11, labelpad=10)
    ax1.set_zlabel('Competitive Ratio', fontsize=11, labelpad=10)
    ax1.set_title(f'CR Surface: Q_{quota_idx+1}, {k_pot_idx+1}-potential',
                  fontsize=13, fontweight='bold', pad=20)
    ax1.view_init(elev=25, azim=45)
    
    # Plot 2: Contour plot (top-down view)
    ax2 = fig.add_subplot(122)
    contour = ax2.contourf(Alpha, B_mesh, CR_surface, levels=15, cmap='viridis')
    contour_lines = ax2.contour(Alpha, B_mesh, CR_surface, levels=15, 
                                 colors='black', alpha=0.3, linewidths=0.5)
    ax2.clabel(contour_lines, inline=True, fontsize=8, fmt='%.3f')
    
    ax2.set_xlabel('Interference Parameter α', fontsize=11)
    ax2.set_ylabel('Budget B', fontsize=11)
    ax2.set_title('CR Contour Plot', fontsize=13, fontweight='bold')
    fig.colorbar(contour, ax=ax2, label='Competitive Ratio')
    
    plt.tight_layout()
    filename = f'k-sec-3d-surface-Q{quota_idx+1}-k{k_pot_idx+1}-Bmax{B_max}.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    print(f"✓ Plot saved: {filename}")
    plt.show()
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"SUMMARY STATISTICS")
    print(f"{'='*60}")
    print(f"B range:           [{min(B_range)}, {max(B_range)}]")
    print(f"α range:           [0.00, 1.00]")
    print(f"CR range:          [{CR_surface.min():.6f}, {CR_surface.max():.6f}]")
    print(f"V_best:            {v_best:.6f}")
    for B in B_range:
        print(f"V_worst(B={B}):       {v_worst_by_B[B]:.6f}")
    print(f"{'='*60}")

run_surface_button.on_click(run_surface_analysis)

In [27]:
# Display 3D surface interface
print("="*60)
print("3D SURFACE ANALYSIS: CR(B, α)")
print("="*60)
print("\nVisualize the complete performance landscape")
print("across both budget B and interference α.\n")

display(widgets.VBox([
    widgets.HTML("<b>Select configuration for 3D surface:</b>"),
    widgets.HBox([surface_quota_widget, surface_k_potential_widget]),
    widgets.HBox([surface_B_max_widget, surface_alpha_step_widget]),
    run_surface_button,
    surface_output_widget
], layout=widgets.Layout(padding='10px')))

3D SURFACE ANALYSIS: CR(B, α)

Visualize the complete performance landscape
across both budget B and interference α.

